In [ ]:
import mlflow
from feast import FeatureStore
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize Feast store
store = FeatureStore(repo_path="./feature_store")

# Get features from Feast
features = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "mushroom_features:cap_shape",
        "mushroom_features:cap_surface",
        "mushroom_features:cap_color",
        # Add other features here
    ],
).to_df()

# Prepare data
X = features.drop("class", axis=1)
y = features["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Start MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("model", "RandomForestClassifier")
    mlflow.log_param("features", list(X.columns))
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Make predictions and calculate accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")

print(f"Model accuracy: {accuracy}")